##### In 1.Chatbot we were using ImagineClient as our LLM. ImagineClient is an LLM that is made by qualcomm and cannot be used like a LLM model in langchain or langgraph for that part. So after going through the docs I figured out that qualcomm playgrounds actually has a different modeltype for using as an LLM Model for langchain called ImagineChat. So please do keep this in mind that for general purpose we are going to use ImagineClient and for langchain purpose we are going to use ImagineChat

In [3]:
#Importing libraries
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["IMAGINE_API_KEY"]=os.getenv("IMAGINE_API_KEY")
os.environ["IMAGINE_API_ENDPOINT"]=os.getenv("IMAGINE_ENDPOINT_URL")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate(
    [
        ("system","You are a an expert in the field of Software Testing. You have decades of experience in creating test plan and test strategy in the field of software development and embedded software development. Please generate a test strategy based on user requirements."),
        ("user","{input}")
    ]
)

In [5]:
from imagine.langchain import ImagineChat
model = ImagineChat(model="Llama-3-8B")
model

ImagineChat(client=<imagine.client.ImagineClient object at 0x0000017F1BEF5FF0>, async_client=<imagine.async_client.ImagineAsyncClient object at 0x0000017F1BEF71F0>, model='Llama-3-8B')

In [7]:
chain = prompt | model
result = chain.invoke({"input":"Generate a test strategy for a generic login page"})
print(result)

ImagineAPITooManyRequestsException: Status: 429. Message: {"message":"Too many requests. Invalid model/rate limits not configured for this model.","status":"error"}

##### For some reason the ImagineChat model is not working like normal Ollama and GROQ model works. So referring  to the documentation to see what is different

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from imagine.langchain import ImagineChat

model = ImagineChat(model="Llama-3-8B")
response = model.invoke(
        [
            SystemMessage(content="Translate the following from English into Italian"),
            HumanMessage(content="hello"),
        ]
    )

print(response.content)

for chunk in model.stream(
    [
        HumanMessage(content="hello!"),
        AIMessage(content="Hi there human!"),
        HumanMessage(
            content="Write a program to sort a list of numbers in python!"
        ),
    ], max_tokens=512):

    print(chunk.content, end="", flush=True)


ImagineAPITooManyRequestsException: Status: 429. Message: {"message":"Too many requests. Invalid model/rate limits not configured for this model.","status":"error"}

##### Ok so the code above belongs to first code written in the docs but still facing the same error. I have also checked the rates at which I can make the requests and certainly I am nowhere close to the limit that they have put per minute let alone per hour. I am trying to figure out why this is happening

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

from imagine.langchain import ImagineChat


model = ImagineChat(model="Llama-3.1-8B", max_tokens=200)
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]

response = model.invoke(messages)

print(response.content)

The primary purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model is too complex and learns the noise in the training data, resulting in poor performance on new, unseen data.

Regularization techniques aim to reduce overfitting by adding a penalty term to the loss function, which discourages the model from learning complex patterns in the data. This is achieved by:

1. **Reducing the capacity of the model**: Regularization techniques, such as L1 (Lasso) and L2 (Ridge) regularization, add a penalty term to the loss function that increases as the model's weights or features become larger. This discourages the model from learning large weights or features, effectively reducing its capacity.
2. **Preventing feature selection**: Regularization techniques can also prevent the model from selecting features that are not relevant to the problem, by adding a penalty term for each feature used.
3. **Improving generalizability**: By r

##### Ok so I figred out why this was happening. The model that you are using in the code that is not working is because you have used Llama 3 instead of Llama3.1

In [11]:
from imagine.langchain import ImagineChat
model = ImagineChat(model="Llama-3.1-8B")
chain = prompt | model
result = chain.invoke({"input":"Generate a test strategy for a generic login page"})
print(result.content)

**Test Plan for Generic Login Page**

**Test Plan ID:** LP-001
**Test Plan Name:** Login Page Test Plan
**Project Name:** Generic Login System
**Version:** 1.0
**Test Plan Owner:** [Your Name]
**Test Plan Date:** [Current Date]

**Overview:**
The purpose of this test plan is to ensure that the generic login page functions as expected, allowing users to successfully log in to the system. The test plan covers various scenarios,


In [12]:
print(result)

content='**Test Plan for Generic Login Page**\n\n**Test Plan ID:** LP-001\n**Test Plan Name:** Login Page Test Plan\n**Project Name:** Generic Login System\n**Version:** 1.0\n**Test Plan Owner:** [Your Name]\n**Test Plan Date:** [Current Date]\n\n**Overview:**\nThe purpose of this test plan is to ensure that the generic login page functions as expected, allowing users to successfully log in to the system. The test plan covers various scenarios,' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 71, 'total_tokens': 171, 'completion_tokens': 100}, 'model': 'Llama-3.1-8B', 'finish_reason': 'FinishReason.stop'} id='run-8d99e880-0ed3-47f3-aa3f-7cf351762b65-0' usage_metadata={'input_tokens': 71, 'output_tokens': 100, 'total_tokens': 171}


In [ ]:
result2 = chain.invoke({"input":"Explain to me in 1000 words how to generate a test strategy and test plan"})
print(result.content)